In [1]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [2]:
data = pd.read_csv('dataset.csv')

/var/folders/xw/6y1gg0b108956d53nhdcltr00000gn/T/ipykernel_69159/3991704923.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('dataset.csv')


In [3]:
data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,8522718,ROICVZB7S1LHH,B00S5OY978,919214172,BB Dakota Women's Danica Lace Racerback Dress,Apparel,4.0,1.0,1.0,N,Y,"Nice dress, but false adverts","Nice dress, but false adverts, the dress u sho...",2015-07-07
1,US,21070679,RP1R4QY0MRGIH,B00RX0I7BE,555815695,Cute Unisex Baby Crochet Cap Beanie with Suspe...,Apparel,4.0,0.0,0.0,N,Y,But was a nice prop. Loved it,It was on the big size. But was a nice prop. L...,2015-08-02
2,US,11912960,R4555MF1I56JR,B007SOMDV0,850882879,Sakkas Stonewashed Rayon Embroidered Adjustabl...,Apparel,5.0,0.0,0.0,N,Y,Gorgeous!,Oh how I lOVE this dress! So gorgeous and in a...,2015-07-22
3,US,14600978,R3F6R1I5S97QGC,B00OIC9QF6,924232065,Zeagoo Women's Batwing Sleeve Tassels Hem Cloa...,Apparel,5.0,0.0,0.0,N,Y,Five Stars,Love the sweater. Fit and quality as expected....,2015-04-21
4,US,13901935,R3EITK2YG2C1IG,B008UQFNDA,988402841,Kitestrings Little Boys' Faux Fur Trimmed Hood...,Apparel,4.0,0.0,0.0,N,Y,"Warm, nice looking, fits well!",LOve it! So easy to wipe clean and I love that...,2013-10-29


# Data processing

In [4]:
import re
import pandas as pd
import html
import spacy
from spacy.language import Language
from spacy.tokens import Doc
from typing import List, TypeVar, Callable
from tqdm import tqdm

data['review_body'] = data['review_body'].astype(str)

# Load the spaCy English model
nlp = spacy.load("/Users/nirmitsachde/Library/CloudStorage/GoogleDrive-sachde.n@husky.neu.edu/My Drive/DAE/DME/project/amazon-reviews-nlp-sentiment-analysis-main/venv/lib/python3.9/site-packages/en_core_web_md/en_core_web_md-3.6.0")

def lower_text(text: str) -> str:
    return text.lower()

def strip_text(text: str) -> str:
    return text.strip()

def unescape_html(text: str) -> str:
    return html.unescape(text)

def remove_punctuation(text: str) -> str:
    return re.sub(r"[^\w\s]", "", text)

def remove_special_characters(text: str) -> str:
    return re.sub("[^a-zA-Z0-9\s]", "", text)

def remove_html_tags(text: str) -> str:
    return re.sub("<.*?>", "", text)

PRE_NLP_STRING_PREPROCESS_FUNCTION = TypeVar("PRE_NLP_STRING_PREPROCESS_FUNCTION", bound="Callable[[str], str]")

DEFAULT_PRE_NLP_STRING_PREPROCESS_FUNCTION: List[PRE_NLP_STRING_PREPROCESS_FUNCTION] = [
    lower_text,
    strip_text,
    remove_special_characters,
    remove_punctuation,
    unescape_html,
    remove_html_tags,
]

def remove_stopwords(nlp: Language, text: str) -> str:
    doc = nlp(text)
    tokens = []

    for token in doc:
        if not token.is_stop:
            tokens.append(token.text)

    return " ".join(tokens)

def lemmatize(nlp: Language, text: str) -> str:
    doc = nlp(text)
    tokens = []

    for token in doc:
        tokens.append(token.lemma_)

    return " ".join(tokens)

def lemmatize_and_remove_stopwords(doc: Doc) -> str:
    tokens = []

    for token in doc:
        if not token.is_stop:
            tokens.append(token.lemma_)

    return " ".join(tokens)

def run_batch_text_normalization_pipeline(nlp: Language, texts: pd.Series, pre_nlp_string_functions: List[PRE_NLP_STRING_PREPROCESS_FUNCTION] = DEFAULT_PRE_NLP_STRING_PREPROCESS_FUNCTION, show_batch_progress: bool = False, texts_type: str = "series") -> pd.Series:
    def apply_prenlp_processing_steps(text: str) -> str:
        if isinstance(text, str):
            for pre_nlp_string_function in pre_nlp_string_functions:
                text = pre_nlp_string_function(text)
        return text

    prenlp_texts = texts.apply(apply_prenlp_processing_steps)
    docs = nlp.pipe(texts=prenlp_texts, n_process=-1)
    final_docs = (docs if not show_batch_progress else tqdm(docs, total=prenlp_texts.shape[0]))
    texts: List[str] = []

    for doc in final_docs:
        texts.append(lemmatize_and_remove_stopwords(doc))

    if texts_type == "series":
        return pd.Series(texts)

    return texts
# Assuming 'data' is already defined and contains the DataFrame
processed_data = run_batch_text_normalization_pipeline(nlp, data['review_body'])



/Users/nirmitsachde/.local/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [8]:
amazon_reviews = pd.concat([data, processed_data.rename("processed_text")], axis=1)

In [9]:
amazon_reviews.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,processed_text
0,US,8522718,ROICVZB7S1LHH,B00S5OY978,919214172,BB Dakota Women's Danica Lace Racerback Dress,Apparel,4.0,1.0,1.0,N,Y,"Nice dress, but false adverts","Nice dress, but false adverts, the dress u sho...",2015-07-07,nice dress false advert dress u showing not u ...
1,US,21070679,RP1R4QY0MRGIH,B00RX0I7BE,555815695,Cute Unisex Baby Crochet Cap Beanie with Suspe...,Apparel,4.0,0.0,0.0,N,Y,But was a nice prop. Loved it,It was on the big size. But was a nice prop. L...,2015-08-02,big size nice prop love
2,US,11912960,R4555MF1I56JR,B007SOMDV0,850882879,Sakkas Stonewashed Rayon Embroidered Adjustabl...,Apparel,5.0,0.0,0.0,N,Y,Gorgeous!,Oh how I lOVE this dress! So gorgeous and in a...,2015-07-22,oh love dress gorgeous rich chocolate color en...
3,US,14600978,R3F6R1I5S97QGC,B00OIC9QF6,924232065,Zeagoo Women's Batwing Sleeve Tassels Hem Cloa...,Apparel,5.0,0.0,0.0,N,Y,Five Stars,Love the sweater. Fit and quality as expected....,2015-04-21,love sweater fit quality expect worth
4,US,13901935,R3EITK2YG2C1IG,B008UQFNDA,988402841,Kitestrings Little Boys' Faux Fur Trimmed Hood...,Apparel,4.0,0.0,0.0,N,Y,"Warm, nice looking, fits well!",LOve it! So easy to wipe clean and I love that...,2013-10-29,love easy wipe clean love lining fleecey warm ...


In [10]:
from tqdm import tqdm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from typing import List, Tuple
import pandas as pd


def get_sentiment_compound_score(
    analyzer: SentimentIntensityAnalyzer, text: str
) -> float:
    senti_scorer = analyzer.polarity_scores(
        text
    ) 
    return senti_scorer["compound"]  


def get_sentiment_label(compound_score: float) -> str:
    if compound_score >= 0.05:
        return "positive"

    if compound_score <= -0.05:
        return "negative"

    return "neutral"


def sscorer(text: str) -> Tuple[float, str]:
    compound_score = get_sentiment_compound_score(text)
    sentiment_label = get_sentiment_label(compound_score)

    return compound_score, sentiment_label


def compute_sentiments_in_batch(
    analyzer: SentimentIntensityAnalyzer,
    texts: pd.Series,
    show_batch_progress: bool = False,
) -> Tuple[List[float], List[str]]:
    compound_scores: List[float] = []
    sentiment_labels: List[str] = []

    for text in tqdm(texts, total=texts.shape[0]) if show_batch_progress else texts:
        compound_score = get_sentiment_compound_score(analyzer, text)
        sentiment_label = get_sentiment_label(compound_score)

        compound_scores.append(compound_score)
        sentiment_labels.append(sentiment_label)

    return compound_scores, sentiment_labels



In [14]:
analyzer = SentimentIntensityAnalyzer()
compound_scores, sentiment_labels = compute_sentiments_in_batch(analyzer, data['review_body'])

# Convert compound_scores and sentiment_labels to Pandas Series if they are not already
compound_scores = pd.Series(compound_scores)
sentiment_labels = pd.Series(sentiment_labels)

# Create a DataFrame for compound_scores and sentiment_labels
sentiment_df = pd.concat([compound_scores, sentiment_labels], axis=1)

# Concatenate sentiment_df with amazon_reviews if amazon_reviews is already a DataFrame
if isinstance(amazon_reviews, pd.DataFrame):
    amazon_reviews = pd.concat([amazon_reviews, sentiment_df], axis=1)
else:
    # If amazon_reviews is a list, you may need to convert it to a DataFrame first
    amazon_reviews = pd.DataFrame(amazon_reviews)
    amazon_reviews = pd.concat([amazon_reviews, sentiment_df], axis=1)


In [13]:
amazon_reviews.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,processed_text
0,US,8522718,ROICVZB7S1LHH,B00S5OY978,919214172,BB Dakota Women's Danica Lace Racerback Dress,Apparel,4.0,1.0,1.0,N,Y,"Nice dress, but false adverts","Nice dress, but false adverts, the dress u sho...",2015-07-07,nice dress false advert dress u showing not u ...
1,US,21070679,RP1R4QY0MRGIH,B00RX0I7BE,555815695,Cute Unisex Baby Crochet Cap Beanie with Suspe...,Apparel,4.0,0.0,0.0,N,Y,But was a nice prop. Loved it,It was on the big size. But was a nice prop. L...,2015-08-02,big size nice prop love
2,US,11912960,R4555MF1I56JR,B007SOMDV0,850882879,Sakkas Stonewashed Rayon Embroidered Adjustabl...,Apparel,5.0,0.0,0.0,N,Y,Gorgeous!,Oh how I lOVE this dress! So gorgeous and in a...,2015-07-22,oh love dress gorgeous rich chocolate color en...
3,US,14600978,R3F6R1I5S97QGC,B00OIC9QF6,924232065,Zeagoo Women's Batwing Sleeve Tassels Hem Cloa...,Apparel,5.0,0.0,0.0,N,Y,Five Stars,Love the sweater. Fit and quality as expected....,2015-04-21,love sweater fit quality expect worth
4,US,13901935,R3EITK2YG2C1IG,B008UQFNDA,988402841,Kitestrings Little Boys' Faux Fur Trimmed Hood...,Apparel,4.0,0.0,0.0,N,Y,"Warm, nice looking, fits well!",LOve it! So easy to wipe clean and I love that...,2013-10-29,love easy wipe clean love lining fleecey warm ...


In [16]:
import os
import spacy
import pandas as pd
from dotenv import load_dotenv
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from typing import List

load_dotenv()
PANDAS_SAMPLE_LIMIT = int(os.getenv("PANDAS_SAMPLE_LIMIT", 95_000))

# Define the path to the directory containing the en_core_web_md model data
MODEL_PATH = "/Users/nirmitsachde/Library/CloudStorage/GoogleDrive-sachde.n@husky.neu.edu/My Drive/DAE/DME/project/amazon-reviews-nlp-sentiment-analysis-main/venv/lib/python3.9/site-packages/en_core_web_md/en_core_web_md-3.6.0"

def process_amazon_reviews(data: pd.DataFrame) -> pd.DataFrame:
    amazon_reviews = data.copy()

    # Load the spaCy model
    nlp_model = spacy.load(MODEL_PATH, disable=["parser", "ner", "tok2vec"])
    sentiment_analyzer = SentimentIntensityAnalyzer()

    # Verify if the required columns exist in the DataFrame
    if 'review_body' not in amazon_reviews.columns or 'star_rating' not in amazon_reviews.columns:
        print("Skipping dataset because 'review_body' or 'star_rating' column is missing.")
        return None

    # Drop rows with missing values in the required columns
    amazon_reviews = amazon_reviews.dropna(subset=["review_body", "star_rating"])

    rows = amazon_reviews.shape[0]
    amazon_reviews_samples = amazon_reviews.sample(
        n=PANDAS_SAMPLE_LIMIT if rows > PANDAS_SAMPLE_LIMIT else rows
    )

    print("Finished processing the dataset.\n")

    return amazon_reviews_samples

if __name__ == "__main__":
    # Assuming 'data' is already defined and contains the DataFrame
    amazon_reviews = process_amazon_reviews(amazon_reviews)


Finished processing the dataset.



In [15]:
amazon_reviews.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,processed_text,0,1
0,US,8522718,ROICVZB7S1LHH,B00S5OY978,919214172,BB Dakota Women's Danica Lace Racerback Dress,Apparel,4.0,1.0,1.0,N,Y,"Nice dress, but false adverts","Nice dress, but false adverts, the dress u sho...",2015-07-07,nice dress false advert dress u showing not u ...,-0.6946,negative
1,US,21070679,RP1R4QY0MRGIH,B00RX0I7BE,555815695,Cute Unisex Baby Crochet Cap Beanie with Suspe...,Apparel,4.0,0.0,0.0,N,Y,But was a nice prop. Loved it,It was on the big size. But was a nice prop. L...,2015-08-02,big size nice prop love,0.8765,positive
2,US,11912960,R4555MF1I56JR,B007SOMDV0,850882879,Sakkas Stonewashed Rayon Embroidered Adjustabl...,Apparel,5.0,0.0,0.0,N,Y,Gorgeous!,Oh how I lOVE this dress! So gorgeous and in a...,2015-07-22,oh love dress gorgeous rich chocolate color en...,0.9578,positive
3,US,14600978,R3F6R1I5S97QGC,B00OIC9QF6,924232065,Zeagoo Women's Batwing Sleeve Tassels Hem Cloa...,Apparel,5.0,0.0,0.0,N,Y,Five Stars,Love the sweater. Fit and quality as expected....,2015-04-21,love sweater fit quality expect worth,0.8658,positive
4,US,13901935,R3EITK2YG2C1IG,B008UQFNDA,988402841,Kitestrings Little Boys' Faux Fur Trimmed Hood...,Apparel,4.0,0.0,0.0,N,Y,"Warm, nice looking, fits well!",LOve it! So easy to wipe clean and I love that...,2013-10-29,love easy wipe clean love lining fleecey warm ...,0.9721,positive
